匯入函式庫和資料

In [ ]:
import tensorflow as tf

# 顯示 TensorFlow 版本
print("TensorFlow version:", tf.__version__)

# 列出可用的實體裝置
from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()

print("\nAvailable devices:")
for device in devices:
    print(f"- {device.name} ({device.device_type})")

In [ ]:
import os
import glob

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mimg
%matplotlib inline
import cv2

from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras import layers, Model
from sklearn.model_selection import train_test_split

In [ ]:
"""
from google.colab import drive
drive.mount('/content/drive')
"""

In [ ]:
INPUT_PATH = "./data/realwaste-main/RealWaste"
print(os.listdir(INPUT_PATH))

In [ ]:
glass = Path(INPUT_PATH + '/Glass').glob('*.jpg')
metal = Path(INPUT_PATH + '/Metal').glob('*.jpg')
foodorga = Path(INPUT_PATH + '/Food Organics').glob('*.jpg')
mistrash = Path(INPUT_PATH + '/Miscellaneous Trash').glob('*.jpg')
plastic = Path(INPUT_PATH + '/Plastic').glob('*.jpg')
paper = Path(INPUT_PATH + '/Paper').glob('*.jpg')
textrash = Path(INPUT_PATH + '/Textile Trash').glob('*.jpg')
cardboard = Path(INPUT_PATH + '/Cardboard').glob('*.jpg')
vegetation = Path(INPUT_PATH + '/Vegetation').glob('*.jpg')

In [ ]:
galss_data = [(image, 0) for image in glass]
metal_data = [(image, 1) for image in metal]
foodorga_data = [(image, 2) for image in foodorga]
mistrash_data = [(image, 3) for image in mistrash]
plastic_data = [(image, 4) for image in plastic]
paper_data = [(image, 5) for image in paper]
textrash_data = [(image, 6) for image in textrash]
cardboard_data = [(image, 7) for image in cardboard]
vegetation_data = [(image, 8) for image in vegetation]

total_data = galss_data + metal_data + foodorga_data + mistrash_data + plastic_data + paper_data + textrash_data + cardboard_data + vegetation_data
total_data = pd.DataFrame(total_data, columns=['image', 'label'])

In [ ]:
train_val_df, test_df = train_test_split(total_data, test_size=0.10, random_state=42)

In [ ]:
train_df, val_df = train_test_split(train_val_df, test_size=1/6, random_state=42)

訓練資料集處理

In [ ]:
train_df = train_df.sample(frac=1., random_state=100).reset_index(drop=True)

In [ ]:
count_result = train_df['label'].value_counts()
print('Total : ', len(train_df))
print(count_result)

# Plot the results
plt.figure(figsize=(24,5))
sns.countplot(x = 'label', data =  train_df)
plt.title('Number of classes', fontsize=16)
plt.xlabel('Class type', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(range(len(count_result.index)),
           ['Glass : 0', 'Metal : 1', 'Food Organics : 2', 'Miscellaneous Trash : 3', 'Plastic : 4', 'Paper : 5', 'Textile Trash : 6', 'Cardboard : 7', 'Vegetation : 8'],
           fontsize=14)
plt.show()

In [ ]:
def data_input(dataset):
    #print(dataset.shape)
    for image in dataset:
        im = cv2.imread(str(image))
        im = cv2.resize(im, (224,224))
        if im.shape[2] == 1:
          im = np.dstack([im, im, im])
        x_image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        x_image = x_image.astype(np.float32)/255.
        return x_image

In [ ]:
x_train, y_train = ([data_input(train_df.iloc[i][:]) for i in range(len(train_df))],
            [train_df.iloc[i][1] for i in range(len(train_df))])
x_train = np.array(x_train)
y_train = np.array(y_train)

print("Total number of validation examples: ", x_train.shape)
print("Total number of labels:", y_train.shape)

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train,9)

In [ ]:
fig, ax = plt.subplots(3, 4, figsize=(20,15))
for i, axi in enumerate(ax.flat):
    image = imread(train_df.image[i])
    axi.imshow(image, cmap='bone')
    axi.set_title(('Glass' if train_df.label[i] == 0
                   else 'Metal' if train_df.label[i] == 1
                   else 'Food Organics' if train_df.label[i] == 2
                   else 'Miscellaneous Trash' if train_df.label[i] == 3
                   else 'Plastic' if train_df.label[i] == 4
                   else 'Paper' if train_df.label[i] == 5
                   else 'Textile Trash' if train_df.label[i] == 6
                   else 'Cardboard' if train_df.label[i] == 7
                   else 'Vegetation')
                  + '  [size=' + str(image.shape) +']',
                  fontsize=14)
    axi.set(xticks=[], yticks=[])

Validation資料集處理

In [ ]:
val_df = val_df.sample(frac=1., random_state=100).reset_index(drop=True)

In [ ]:
count_result = val_df['label'].value_counts()
print('Total : ', len(val_df))
print(count_result)

# Plot the results
plt.figure(figsize=(24,5))
sns.countplot(x = 'label', data =  val_df)
plt.title('Number of classes', fontsize=16)
plt.xlabel('Class type', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(range(len(count_result.index)),
           ['Glass : 0', 'Metal : 1', 'Food Organics : 2', 'Miscellaneous Trash : 3', 'Plastic : 4', 'Paper : 5', 'Textile Trash : 6', 'Cardboard : 7', 'Vegetation : 8'],
           fontsize=14)
plt.show()

In [ ]:
x_val, y_val = ([data_input(val_df.iloc[i][:]) for i in range(len(val_df))],
            [val_df.iloc[i][1] for i in range(len(val_df))])
x_val = np.array(x_val)
y_val = np.array(y_val)

print("Total number of validation examples: ", x_val.shape)
print("Total number of labels:", y_val.shape)

In [ ]:
y_val = tf.keras.utils.to_categorical(y_val,9)

測試資料集處理

In [ ]:
test_df = test_df.sample(frac=1., random_state=100).reset_index(drop=True)

In [ ]:
count_result = test_df['label'].value_counts()
print('Total : ', len(test_df))
print(count_result)

# Plot the results
plt.figure(figsize=(24,5))
sns.countplot(x = 'label', data =  test_df)
plt.title('Number of classes', fontsize=16)
plt.xlabel('Class type', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(range(len(count_result.index)),
           ['Glass : 0', 'Metal : 1', 'Food Organics : 2', 'Miscellaneous Trash : 3', 'Plastic : 4', 'Paper : 5', 'Textile Trash : 6', 'Cardboard : 7', 'Vegetation : 8'],
           fontsize=14)
plt.show()

In [ ]:
x_test, y_test = ([data_input(test_df.iloc[i][:]) for i in range(len(test_df))],
            [test_df.iloc[i][1] for i in range(len(test_df))])
x_test = np.array(x_test)
y_test = np.array(y_test)

print("Total number of validation examples: ", x_test.shape)
print("Total number of labels:", y_test.shape)

In [ ]:
y_test = tf.keras.utils.to_categorical(y_test,9)

In [ ]:
IMG_SIZE = (224,224)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                        include_top=False,
                        weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(16, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(9, activation = 'softmax')(x)
model = Model(base_model.input,x)

In [ ]:
base_learning_rate = 0.001

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
batch_size = 16
initial_epochs = 50

In [ ]:
history = model.fit(x_train, y_train,
          batch_size = batch_size,
          epochs = initial_epochs,
          validation_data = (x_val, y_val)
          )

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy :', accuracy)